In [1]:
import pandas as pd
from thefuzz import fuzz
from thefuzz import process
from unidecode import unidecode
import nltk
import hazm
import string
import re

In [2]:
df_all=pd.read_csv('D:\quera\\bootcamp\examples\Laptop_analysis_and_Warehousing_g3\Data Source\\all_data_c.csv')
df_sel=pd.read_csv('D:\quera\\bootcamp\examples\Laptop_analysis_and_Warehousing_g3\Data Source\sales_data\sales_data.csv')

In [3]:
df_all=df_all.dropna()
df_sel=df_sel.dropna()

In [4]:
idx=df_sel[['Manufacturer',
       'Model_Name', 'Category', 'Screen_Size', 'Screen', 'CPU', 'RAM',
       'Storage', 'GPU', 'OS', 'OS_Version', 'Weight']].drop_duplicates().index

In [5]:
idx1=df_all[['Id','Attributes']].drop_duplicates().index

In [6]:
df_sel=df_sel.loc[idx]
df_all=df_all.loc[idx1]

In [7]:
df_sel['text']=df_sel['Manufacturer']+' '+df_sel['Model_Name']+' '+df_sel['Screen_Size']+' '+df_sel['CPU']+' '+df_sel['RAM']+' '+df_sel['Storage']+' '+df_sel['GPU']

In [8]:
df_all['text']=df_all.Title

In [9]:
def remove_ir(text):
    t=[]
    tokens = text.split(' ')
    for i in range(len(tokens)):
        t.append(''.join(re.findall(r'[0-9َA-Za-z۰-۹]', tokens[i])))
    return "  ".join(t)
df_sel['text'] = df_sel['text'].apply(lambda x: remove_ir(x))
df_all['text'] = df_all['text'].apply(lambda x: remove_ir(x))

In [10]:
def convert_to_lower(text):
    return text.lower()
df_sel['text'] = df_sel['text'].apply(lambda x: convert_to_lower(x))
df_all['text'] = df_all['text'].apply(lambda x: convert_to_lower(x))

In [11]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))
df_sel['text'] = df_sel['text'].apply(lambda x: remove_punctuation(x))
df_all['text'] = df_all['text'].apply(lambda x: remove_punctuation(x))

In [12]:
with open('stopwords.txt',encoding='utf-8') as stopwords_file:
    stopwords=stopwords_file.readlines()
stopwords=[line.replace('\n','') for line in stopwords]
nltk.download('stopwords')
nltk_stopwords=nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aghil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def remove_stopwords(text):
    removed = []
    stop_words = stopwords
    tokens = nltk.word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] not in stop_words:
            removed.append(tokens[i])
    return " ".join(removed)
df_sel['text'] = df_sel['text'].apply(lambda x: remove_stopwords(x))
df_all['text'] = df_all['text'].apply(lambda x: remove_stopwords(x))

In [14]:
def remove_extra_white_spaces(text):
    single_char_pattern = r'\s+[a-zA-Z0-9آ-ی۰-۹]\s+'
    without_sc = re.sub(pattern=single_char_pattern, repl=" ", string=text)
    return without_sc
df_sel['text'] = df_sel['text'].apply(lambda x: remove_extra_white_spaces(x))
df_all['text'] = df_all['text'].apply(lambda x: remove_extra_white_spaces(x))

In [15]:
def stemmering_e(text):
    stemmer = nltk.SnowballStemmer('english')
    tokens = nltk.word_tokenize(text)
    for i in range(len(tokens)):
        lemma_word = stemmer.stem(tokens[i])
        tokens[i] = lemma_word
    return " ".join(tokens)
df_sel['text'] = df_sel['text'].apply(lambda x: stemmering_e(x))
df_all['text'] = df_all['text'].apply(lambda x: stemmering_e(x))

In [16]:
def unicode(text):
    return unidecode(text)
df_sel['text'] = df_sel['text'].apply(lambda x: unicode(x))
df_all['text'] = df_all['text'].apply(lambda x: unicode(x))

In [17]:
df_sel.text.values[100]

'dell vostro 5568 156 intel core i5 7200u 25ghz 8gb 256gb ssd intel hd graphic 620'

In [21]:
pr=[]
for index,row in df_sel.iterrows():
    for index_base,row_base in df_all.iterrows():
        #print(fuzz.token_sort_ratio(row_base['text'], row['text']))
        if fuzz.token_sort_ratio(row_base['text'], row['text'])<20:
            pr.append(row_base['Id'])

In [22]:
len(set(pr))

136